In [59]:
import pandas as pd
import requests
import os
from unidecode import unidecode
import json

In [60]:
df = pd.read_csv("missing_gender.csv")

In [61]:
df.head()

,id,person,character,movie,name_id,gender
0,/name/nm13818646/Self,Kenneth Scheuvront,Self,1000_On_The_Trail_of_UFOS_Dark_Sky,nm13818646,NaN
1,/name/nm10541807/Self,Shannon Legro,Self,1000_On_The_Trail_of_UFOS_Dark_Sky,nm10541807,NaN
2,/name/nm13818650/Self,Dave Roberts,Self,1000_On_The_Trail_of_UFOS_Dark_Sky,nm13818650,NaN
3,/name/nm13818652/Self,Steve Hummel,Self,1000_On_The_Trail_of_UFOS_Dark_Sky,nm13818652,NaN
4,/name/nm13818651/Self,Matt Welsch,Self,1000_On_The_Trail_of_UFOS_Dark_Sky,nm13818651,NaN


In [62]:
df = df[~df['name_id'].isnull()]

In [65]:
try:
    with open("genderize_data.json", "r") as f:
        out = json.load(f)
except FileNotFoundError:
    out = []

name_list = list(map(lambda x: x['name_id'], out))
df_sub = df[~df['name_id'].isin(name_list)]

for index, row in df_sub.iterrows():
    person = row['person']

    f_name = person.split()[0]

    res = requests.get(f"https://api.genderize.io?name={f_name}")
    
    content = res.json()
    if 'error' in content:
        print(content['error'])
        break
    
    content['name_id'] = row['name_id']

    print(res.status_code, index, row['name_id'], person, content['gender'])
    if res.status_code == 200:
        out.append(content)
    else:
        break

200 1985 nm5077826 Ashley Ingram female
200 1986 nm0072775 Andrew Bentler male
200 1987 nm3619123 Pinto Bennett male
200 1988 nm1318670 Vishal Dadlani male
200 1989 nm1347153 Tyler Perry male
200 1990 nm4519470 Doug Goodwin male
200 1991 nm8620175 Savannah Knoop female
200 1992 nm1380711 Laura Albert female
200 1993 nm1328835 Ira Silverberg female
200 1994 nm0102994 Arthur Bradford male
200 1995 nm0000783 Dario Argento male
200 1996 nm0586965 David Milch male
200 1997 nm6239949 Stephen Beachy male
200 1998 nm0787601 Steven Shainberg male
200 1999 nm2193909 Warren St. John male
200 2000 nm0711840 Brett Ratner male
200 2001 nm0001175 Blake Edwards male
200 2002 nm2692879 Jake Monkarsh male
200 2003 nm11898406 Jennifer Snodgrass female
200 2004 nm0755967 Liz Sagal female
200 2005 nm4621553 Conrad De La Torres III male
200 2006 nm0110786 Thomas H. Brodek male
200 2007 nm1635180 Jeremy Rifkin male
200 2008 nm1670028 Bruce Friedrich male
200 2009 nm6717289 Tara Garnett female
200 2010 nm5245

In [66]:
len(out)

2346

In [67]:
with open("genderize_data.json", "w") as f:
    f.write(json.dumps(out))